In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [1]:
import os, gc
import torch
import pickle
from pathlib import Path
from torch import nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from torch_geometric.nn import global_mean_pool, global_max_pool
from torch_geometric.data import Data, InMemoryDataset, DataLoader
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv, SAGEConv, SGConv
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
np.random.seed(123)

In [2]:
class YooChooseBinaryDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ['click_binary100m.pt']
        # return ['click_binary.pt']    # for whole dataset

    def download(self):
        pass

    def process(self):
        data_list = []
        # clicks = pd.read_csv(root / 'clicks_pro.csv', encoding='utf-8', low_memory=False)  # 使用全部数据
        clicks = pd.read_csv(root / 'clicks_pro_100m.csv', encoding='utf-8', low_memory=False)    # 使用部分数据

        # process by session_id
        grouped = iter(clicks.groupby('session_id'))
        lens = clicks.session_id.unique().shape[0]

        # clicks 不需要常驻内存, 并且使用动态加载
        del clicks

        for session_id, group in tqdm(grouped, total=lens):
            # 重新编码，作为节点的顺序
            sess_item_id = LabelEncoder().fit_transform(group.item_id)

            # 重建索引
            group = group.reset_index(drop=True)
            group['sess_item_id'] = sess_item_id

            # 节点的初始特征
            # 重复的浏览记录当做一次记录
            # 使用 item id 作为节点特征
            # 每个子表 group 都有同样的 session id
            node_features = group.loc[group.session_id == session_id, ['sess_item_id', 'item_id']].sort_values(
                'sess_item_id').item_id.drop_duplicates().values
            node_features = torch.LongTensor(node_features).unsqueeze(1)

            # 序列访问的顺序
            source_nodes = group.sess_item_id.values[:-1]
            target_nodes = group.sess_item_id.values[1:]
            edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)

            x = node_features    # item_id , 可以考虑使用 category
            y = torch.FloatTensor([group.label.values[0]])    # 对graph进行二分类，确定在该session下是否有购买行为

            # 每个 session 当做一个 graph
            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [3]:
class Net(torch.nn.Module):
    def __init__(self, sparse_sz, emb_sz=128, p=0.5):
        super(Net, self).__init__()

        self.item_embedding = nn.Embedding(num_embeddings=sparse_sz, embedding_dim=emb_sz)

        self.conv1 = SAGEConv(emb_sz, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)

        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)

        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)

        self.fc1 = nn.Sequential(nn.Linear(256, 128), nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(128, 64), nn.ReLU())
        self.linear = nn.Linear(64, 1)
        self.drop = nn.Dropout(p)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.item_embedding(x).squeeze(1)

        x = F.relu(self.conv1(x, edge_index))
        x, edge_index, _, batch, *_ = self.pool1(x, edge_index, batch=batch)
        x1 = torch.cat([global_max_pool(x, batch), global_mean_pool(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
        x, edge_index, _, batch, *_ = self.pool2(x, edge_index, batch=batch)
        x2 = torch.cat([global_max_pool(x, batch), global_mean_pool(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))
        x, edge_index, _, batch, *_ = self.pool3(x, edge_index, batch=batch)
        x3 = torch.cat([global_max_pool(x, batch), global_mean_pool(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.fc1(x)
        x = self.fc2(x)
        x = self.drop(x)

        x = torch.sigmoid(self.linear(x)).squeeze(1)

        return x

In [4]:
def train():
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = criterion(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)


@torch.no_grad()
def evaluate(loader):
    model.eval()

    predictions = []
    labels = []

    for data in loader:
        data = data.to(device)
        pred = model(data).detach().cpu().numpy()
        label = data.y.detach().cpu().numpy()
        predictions.append(pred)
        labels.append(label)
    predictions = np.hstack(predictions)
    labels = np.hstack(labels)
    return roc_auc_score(labels, predictions)

In [ ]:
 #  各种设置
lr, bs, ps = 0.001, 512, 0.5
sparse_sz, emb_sz = 37495, 128    # sparse_sz = clicks.item_id.max() + 1
# sparse_sz, emb_sz = 48256, 128    # for whole dataset
num_epochs = 10
root = Path('../data/yoochoose-data/')
# device = torch.device('cuda: 0' if torch.cuda.is_available() else 'cpu')
device = torch.device( 'cpu')
model = Net(sparse_sz, emb_sz=emb_sz, p=ps).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss()

# 数据集相关
dataset = YooChooseBinaryDataset(root=root)
dataset = dataset.shuffle()
train_dataset, val_dataset, test_dataset = dataset[:800000], dataset[800000:900000], dataset[900000:]

# loaders
train_loader = DataLoader(train_dataset, batch_size=bs)
val_loader = DataLoader(val_dataset, batch_size=bs)
test_loader = DataLoader(test_dataset, batch_size=bs)
del dataset, train_dataset, val_dataset, test_dataset

In [ ]:
for epoch in range(num_epochs):
    loss = train()    # 训练

    # 评估
    train_auc = evaluate(train_loader)
    val_auc = evaluate(val_loader)
    test_auc = evaluate(test_loader)
    print(
        f'Epoch: {epoch:03d}, Loss: {loss:.5f}, Train Auc: {train_auc:.5f}, Val Auc: {val_auc:.5f}, Test Auc: {test_auc:.5f}'
    )